In [ ]:
%cd ..\src

In [ ]:
import time

In [ ]:
time.perf_counter_ns()

In [ ]:
from os.path import isfile, join

import wandb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import numpy as np

from configs.utils import get_config, get_int_from_config
from echovpr.models.netvlad_encoder import NetVLADEncorder
from extract_netvlad_repr import get_nordland_datasets

import logging

logging.basicConfig(level=logging.INFO)
log = logging.getLogger()


In [ ]:
config = get_config("configs\\train_mlp_nordland.ini", log)['main']

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
encoder = NetVLADEncorder(config).eval().to(device)

In [ ]:
datasets = get_nordland_datasets(config)

# Select summer dataset
dataset = datasets[1][1]

In [ ]:
batchsize = 5 #get_int_from_config(config, 'dataset_netvlad_processing_batchsize', None)
dataLoader_threads = get_int_from_config(config, 'dataset_netvlad_processing_threads', None)

dataLoader = DataLoader(dataset, num_workers=dataLoader_threads, batch_size=batchsize, shuffle=False)


In [ ]:
with torch.no_grad():

    batch_timings = []
    batch_timings2 = []
    total_size = 0
    for x in dataLoader:
        
        x = x.to(device)
        
        start = time.perf_counter()

        _ = encoder(x)

        stop = time.perf_counter()

        batch_timings.append(encoder.batch_processing_time)
        batch_timings2.append(stop - start)

In [ ]:
(np.sum(batch_timings, dtype=np.float64) / len(dataset)) * 1000

In [ ]:
(np.sum(batch_timings2, dtype=np.float64) / len(dataset)) * 1000

In [ ]:
# 1 = 6484.6732386198901
# 0.066434133806900552699
# 0.0064846732386198901663

# 0.18184950710350825
# 0.45114349811539578